In [4]:
import pandas as pd

In [5]:
df = pd.read_csv('questionario_post_pulizia.csv')

In [6]:
df

,orario,eta,dispositivo,usa_mezzi,freq_mezzi,provincia_mezzi,che_mezzi,disservizi,applicazioni,affidabilita_app,interesse_crowdsource,interesse_partecipazione,interesse_posizione,preferenza_salita,interesse_classifica,interesse_sblocco_feature,interesse_sblocco_custom,interesse_donazioni,suggerimenti
0,01/05/2023 15.47.08,18-25,Android,Si,Almeno una volta a settimana,Milano,"Tram, Metro, Bus","Ritardi, Irregolarità del servizio, Mancanza d...",Google Maps,4.0,5.0,4.0,Si,Scannerizzando un QR code sul mezzo,5.0,1.0,0.0,4.0,NaN
1,01/05/2023 15.47.20,18-25,iOS,Si,Almeno una volta a settimana,Roma,"Treno, Tram, Bus, Metro","Ritardi, Irregolarità del servizio, Mancanza d...","Moovit, Google Maps, Maps (Apple), App Cotral",3.0,5.0,3.0,Si,Scannerizzando un QR code sul mezzo,2.0,3.0,1.0,5.0,NaN
2,01/05/2023 15.47.24,18-25,Android,Si,Almeno una volta al mese,Napoli,"Treno, Bus, Metro, Tram","Ritardi, Mancanza di copertura, Sovraffollamen...",Google Maps,5.0,5.0,5.0,No,NaN,0.0,1.0,0.0,5.0,NaN
3,01/05/2023 15.47.56,25-35,Android,Si,Almeno una volta al giorno,Bologna,"Metro, Tram, Treno, Bus","Mancanza di copertura, Scarsa accessibilità, S...","Moovit, Google Maps",4.0,5.0,4.0,Si,Tramite un bottone in-app,0.0,0.0,0.0,1.0,NaN
4,01/05/2023 15.49.28,18-25,iOS,Si,Almeno una volta al mese,Napoli,Metro,"Ritardi, Irregolarità del servizio, Sovraffoll...","Google Maps, Maps (Apple)",4.0,5.0,4.0,Si,Tramite un bottone in-app,5.0,4.0,4.0,5.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115,02/05/2023 10.24.13,18-25,iOS,Si,Almeno una volta al giorno,Roma,"Metro, Tram, Treno, Bus","Irregolarità del servizio, Mancanza di copertu...",Google Maps,4.0,4.0,5.0,Si,"Tramite un bottone in-app, Scannerizzando un Q...",1.0,2.0,2.0,4.0,NaN
116,02/05/2023 11.57.10,18-25,Android,Si,Almeno una volta a settimana,Roma,"Bus, Metro","Irregolarità del servizio, Mancanza di copertura","Waze, Google Maps",5.0,2.0,1.0,Si,Automaticamente grazie alla geolocalizzazione,2.0,2.0,1.0,3.0,NaN
117,02/05/2023 12.04.19,18-25,Android,Si,Almeno una volta al giorno,Roma,"Bus, Tram, Metro","Ritardi, Irregolarità del servizio, Mancanza d...","Google Maps, ProBus, My Cicero (MooneyGo)",3.0,4.0,3.0,No,NaN,3.0,3.0,5.0,4.0,NaN
118,02/05/2023 12.06.32,18-25,iOS,Si,Meno di una volta al mese,Roma,Metro,"Ritardi, Irregolarità del servizio, Sovraffoll...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
